# Comparison between GP and GE on a 4bit parity problem


You can find the csv file of parity4 for GE here: https://drive.google.com/file/d/10T_UQT9iQIep0j64loBQRU0XKI07-CTP/view?usp=sharing

Now that we understand what's going on at the single-run level, let's modify our code to do multiple runs. Notice the similarity between this code and the sample code for GA with multiple runs.

First, let's widen our display.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
!pip install deap

Import our tools.

In [3]:
import operator
import math
import random

import numpy

import deap.algorithms as gp_algorithm
from deap import base
from deap import creator
from deap import tools
from deap import gp


import matplotlib.pyplot as plt

import itertools
import networkx as nx



Set our Genetic Programming parameters, one of which is now the number of runs.

In [4]:
# Genetic Programming constants:
HALL_OF_FAME_SIZE = 1
N_RUNS = 5

Set the random seed.

In [5]:
RANDOM_SEED = 412
random.seed(RANDOM_SEED)

GP-Specific constants.

In [6]:
MIN_TREE_HEIGHT = 3
MAX_TREE_HEIGHT = 5
LIMIT_TREE_HEIGHT = 17
MUT_MIN_TREE_HEIGHT = 0
MUT_MAX_TREE_HEIGHT = 2

Some problem constants.

In [7]:
NUM_INPUTS = 4
NUM_COMBINATIONS = 2 ** NUM_INPUTS

Here's our fitness function. This is a bit more involved than before as we first create all our test cases and then write a function to calculate the party error. We declare a wrapper function, **getCost** to make this code more resuable. This way we can change the fitness function and we only need to change the name in **getCost**.

In [8]:
# calculate the truth table of even parity check:
parityIn = list(itertools.product([0, 1], repeat=NUM_INPUTS))
parityOut = []
for row in parityIn:
    parityOut.append(sum(row) % 2)

# calculate the difference between the results of the
# generated function and the expected parity results:
def parityError(individual):
    func = toolbox_gp.compile(expr=individual)
    return sum(func(*pIn) != pOut for pIn, pOut in zip(parityIn, parityOut))

# fitness measure:
def getCost(individual):
    return parityError(individual), # return a tuple

Add our functions and terminals. This time we are using Boolean operators and values.

In [9]:
def nand_(a, b):
    return 1 if not (bool(a) and bool(b)) else 0

def nor_(a, b):
    return 1 if not (bool(a) or bool(b)) else 0

# create the primitive set:
primitiveSet = gp.PrimitiveSet("main", NUM_INPUTS, "in_")
primitiveSet.addPrimitive(operator.and_, 2)
primitiveSet.addPrimitive(operator.or_, 2)
primitiveSet.addPrimitive(nand_, 2)
primitiveSet.addPrimitive(nor_, 2)

# add terminal values:
primitiveSet.addTerminal(1)
primitiveSet.addTerminal(0)

Create our toolbox. This is very similar to the Symbolic Regression notebook except we are using the parameters declared up above.

In [10]:
toolbox_gp = base.Toolbox()

# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# create the Individual class based on the primitive tree:
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

# create a helper function for creating random trees using the primitive set:
toolbox_gp.register("expr", gp.genFull, pset=primitiveSet, min_=MIN_TREE_HEIGHT, max_=MAX_TREE_HEIGHT)

# create the individual operator to fill up an Individual instance:
toolbox_gp.register("individualCreator", tools.initIterate, creator.Individual, toolbox_gp.expr)

# create the population operator to generate a list of individuals:
toolbox_gp.register("populationCreator", tools.initRepeat, list, toolbox_gp.individualCreator)

# create an operator to compile the primitive tree into python code:
toolbox_gp.register("compile", gp.compile, pset=primitiveSet)

toolbox_gp.register("evaluate", getCost)

# genetic operators:
toolbox_gp.register("select", tools.selTournament, tournsize=2)
toolbox_gp.register("mate", gp.cxOnePoint)
toolbox_gp.register("expr_mut", gp.genGrow, min_=MUT_MIN_TREE_HEIGHT, max_=MUT_MAX_TREE_HEIGHT)
toolbox_gp.register("mutate", gp.mutUniform, expr=toolbox_gp.expr_mut, pset=primitiveSet)

# bloat control:
toolbox_gp.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=LIMIT_TREE_HEIGHT))
toolbox_gp.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=LIMIT_TREE_HEIGHT))



Create our statistics. These are a bit more complex than the GA ones because we want to keep track of fitness and size for all runs.

In [11]:
maxListFitness = []
avgListFitness = []
minListFitness = []
stdListFitness = []

maxListSize = []
avgListSize = []
minListSize = []
stdListSize = []

Now the magic happens and we run **N_RUNS** times. Always start with a small number of runs and generations to make sure that everything is working properly before you commit to a larger number. That way, if something goes horribly wrong, Python won't replicate it 30 times before giving you back control!

In [12]:
def run_gp(name, pop_size, p_crossover, p_mutation, generations,toolbox):
  print(f"\n-----Running {name} -----")
  best_fitnesses = []
  best_individuals = []
  random.seed(42)
  for r in range(1, N_RUNS+1):
      population = toolbox.populationCreator(n=pop_size)
    # define the hall-of-fame object:
      hof = tools.HallOfFame(HALL_OF_FAME_SIZE)


    # Create our statistics
      stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
      stats_size = tools.Statistics(len)
      mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
      mstats.register("avg", numpy.mean)
      mstats.register("std", numpy.std)
      mstats.register("min", numpy.min)
      mstats.register("max", numpy.max)


    # Which run are we on?
      print("\n\nCurrently on run", r, "of",N_RUNS)



      population, logbook = gp_algorithm.eaSimple(population,
                                                  toolbox,
                                                  cxpb=p_crossover,
                                                  mutpb=p_mutation,
                                                  ngen=generations,
                                                  stats=mstats,
                                                  halloffame=hof,
                                                  verbose=True)

    #maxFitnessValues, meanFitnessValues = logbook.chapters['fitness'].select("min", "avg")
      meanFitnessValues, stdFitnessValues, minFitnessValues, maxFitnessValues  = logbook.chapters['fitness'].select("avg", "std", "min", "max")
      meanSizeValues, stdSizeValues, minSizeValues, maxSizeValues  = logbook.chapters['size'].select("avg", "std", "min", "max")


    # Save statistics for this run:
      avgListFitness.append(meanFitnessValues)
      stdListFitness.append(stdFitnessValues)
      minListFitness.append(minFitnessValues)
      maxListFitness.append(maxFitnessValues)

      avgListSize.append(meanSizeValues)
      stdListSize.append(stdSizeValues)
      minListSize.append(minSizeValues)
      maxListSize.append(maxSizeValues)

    # print info for best solution found:
      best = hof.items[0]
      print("-- Best Individual = ", best)
      print("-- length={}, height={}".format(len(best), best.height))
      print("-- Best Fitness = ", best.fitness.values[0])
      best_fitnesses.append(best.fitness.values[0])
      best_individuals.append(best)
  best_index = best_fitnesses.index(min(best_fitnesses))

# Get the corresponding individual
  best_individual_overall = best_individuals[best_index]
  best_fitness_overall = best_fitnesses[best_index]
  return {
        "best_fitness": best_fitness_overall,
        "best_individuals": best_individual_overall,
        "all_fitness": best_fitnesses
    }




Now we go on the **GE** Implementation

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [14]:
import os.path

PATH = '/content/drive/MyDrive/grape/'
if os.path.exists(PATH):
    print('grape directory already exists')
else:
    %cd /content/drive/MyDrive/
    !git clone https://github.com/UL-BDS/grape.git
    print('Cloning grape in your Drive')

%cd /content/drive/MyDrive/grape/

grape directory already exists
/content/drive/MyDrive/grape


In [15]:
# Suppressing Warnings:
import warnings
warnings.filterwarnings("ignore")

In [16]:

import grape
import algorithms
from functions import not_, and_, or_, nand_, nor_

from os import path
import pandas as pd
import numpy as np
np.NaN = np.nan


Set our Grammatical Evolution parameters.

In [17]:

ELITE_SIZE = 1 #round(0.01*POPULATION_SIZE) #it should be smaller or equal to HALLOFFAME_SIZE
HALLOFFAME_SIZE = 1 #round(0.01*POPULATION_SIZE) #it should be at least 1

CODON_CONSUMPTION = 'lazy'
GENOME_REPRESENTATION = 'list'
MAX_GENOME_LENGTH = None

MAX_INIT_TREE_DEPTH = 15
MIN_INIT_TREE_DEPTH = 4
MAX_TREE_DEPTH = 35
MAX_WRAPS = 1
CODON_SIZE = 500

REPORT_ITEMS = ['gen', 'invalid', 'avg', 'std', 'min', 'max',
                'best_ind_length', 'avg_length',
                'best_ind_nodes', 'avg_nodes',
                'avg_depth',
                'avg_used_codons', 'best_ind_used_codons',
                'selection_time', 'generation_time']


In [18]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

Read datasets and grammars according to the problem (parity4).

In [19]:
problem = 'parity4'
X_train = np.zeros([4,16], dtype=bool)
Y_train = np.zeros([16,], dtype=bool)
data = pd.read_csv("/content/parity4.csv")  # make sure the file is in current directory
for i in range(4):
    for j in range(16):
        X_train[i,j] = data['in_' + str(i)].iloc[j]
for i in range(16):
    Y_train[i] = data['output'].iloc[i]
GRAMMAR_FILE = 'parity4.bnf'


Print data.

In [20]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.colheader_justify', 'center')

display(data)

,in_0,in_1,in_2,in_3,output
0,0,0,0,0,0
1,0,0,1,0,1
2,0,1,0,0,1
3,0,1,1,0,0
4,1,0,0,1,1
5,1,0,1,1,0
6,1,1,0,1,0
7,1,1,1,1,1
8,0,0,0,0,1
9,0,0,1,0,0


Print grammar.

In [21]:
f = open("grammars/" + GRAMMAR_FILE, "r")
print(f.read())
f.close()

<e>  ::=  <op> | <x>
<op> ::=  and_(<e>,<e>)|
          or_(<e>,<e>)|
          nand_(<e>,<e>)|
          nor_(<e>,<e>)
<x> ::=   x[0]|x[1]|x[2]|x[3]


Set up the grammar addressed by GRAMMAR_FILE in the class Grammar.

In [22]:
BNF_GRAMMAR = grape.Grammar(path.join("grammars", GRAMMAR_FILE))

Define as fitness function the number of outputs wrongly predicted.

In [23]:
def fitness_eval(individual, points):
    x = points[0]
    Y = points[1]

    if individual.invalid == True:
        return np.NaN,

    # Evaluate the expression
    try:
        pred = eval(individual.phenotype)
    except (FloatingPointError, ZeroDivisionError, OverflowError,
            MemoryError):
        # FP err can happen through eg overflow (lots of pow/exp calls)
        # ZeroDiv can happen when using unprotected operators
        return np.NaN,
    assert np.isrealobj(pred)

    # The expected outputs are in Y
    n_samples = len(Y)
    compare = np.equal(Y,pred) # Compare the outputs with the expected values
    n_correct_outputs = np.sum(compare)

    fitness = n_samples - n_correct_outputs

    return fitness,

Create the deap toolbox.

Use negative weights in the base.Fitness since we are trying to minimise the fitness.

grape.Individual is a class with the following attributes: phenotype, nodes, depth, used_codons, invalid (True or False), n_wraps and self.structure.

grape.sensible_initialisation initialises a whole population of GE individuals using Sensible Initialisation.

tools.selTournament usef Tournament to select parents.

grape.crossover_onepoint selects crossover points within the used portion of the genome.

Similarly, grape.mutation_int_flip_per_codon performs only over the effective length.

In [24]:
toolbox_ge = base.Toolbox()

# Fitness & individual
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", grape.Individual, fitness=creator.FitnessMin)

# Population
toolbox_ge.register("populationCreator", grape.random_initialisation)

# Evaluation
toolbox_ge.register("evaluate", fitness_eval)
# Tournament selection:
toolbox_ge.register("select", tools.selTournament, tournsize=6)
# Single-point crossover:
toolbox_ge.register("mate", grape.crossover_onepoint)
# Flip-int mutation:
toolbox_ge.register("mutate", grape.mutation_int_flip_per_codon)


We used random initialization instead of sensible because the grammar caused mapping errors

Set the main function and run it.

Set the statistics object regarding the fitness score, but there are other statistics defined internally.

Perform Grammatical Evolution using algorithms.ge_eaSimpleWithElitism, a simple evolutionary algorithm. The differences to the algorithms.ge_eaSimpleWithElitism used by deap are:

We use elitism (if you do not want to use, just set ELITISM_SIZE = 0);
After crossover and mutation, we check the offspring to assure that the max depth is not violated;
We measure many other things to report, such as the generation and the selection times, the number of invalid individuals etc.

In [25]:
def run_ge(setup_name, pop_size, p_crossover, p_mutation, generations,toolbox=toolbox_ge):
    print(f"\n===== Running {setup_name} =====")

    best_fitnesses = []
    best_individuals = []

    random.seed(42)

    for r in range(N_RUNS):
        print(f"\nCurrently on run {r+1} of {N_RUNS}")

        # create initial population
        population = toolbox.populationCreator(
            ind_class=creator.Individual,
            pop_size=int(pop_size),
            bnf_grammar=BNF_GRAMMAR,
            min_init_genome_length=50,
            max_init_genome_length=500,
            max_init_depth=int(MAX_INIT_TREE_DEPTH),
            codon_size=CODON_SIZE,
            codon_consumption=CODON_CONSUMPTION,
            genome_representation=GENOME_REPRESENTATION
        )
        # define the hall-of-fame object:
        hof = tools.HallOfFame(HALLOFFAME_SIZE)
        # prepare the statistics object:
        stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        stats.register("avg", np.nanmean)
        stats.register("std", np.nanstd)
        stats.register("min", np.nanmin)
        stats.register("max", np.nanmax)

        # perform the Grammatical Evolution flow:
        population, logbook = algorithms.ge_eaSimpleWithElitism(
            population,
            toolbox,
            cxpb=p_crossover,
            mutpb=p_mutation,
            ngen=generations,
            elite_size=ELITE_SIZE,
            bnf_grammar=BNF_GRAMMAR,
            codon_size=CODON_SIZE,
            max_tree_depth=MAX_TREE_DEPTH,
            max_genome_length=MAX_GENOME_LENGTH,
            points_train=[X_train, Y_train],
            codon_consumption=CODON_CONSUMPTION,
            report_items=REPORT_ITEMS,
            genome_representation=GENOME_REPRESENTATION,
            stats=stats,
            halloffame=hof,
            verbose=False
        )

        best = hof.items[0]
        print(f"Run {r+1} Best Fitness: {best.fitness.values[0]}")
        print(f"Run {r+1} Best Individual: {best.phenotype}\n")

        best_fitnesses.append(best.fitness.values[0])
        best_individuals.append(best.phenotype)

    # Pick overall best
    best_index = best_fitnesses.index(min(best_fitnesses))
    best_overall = best_individuals[best_index]
    best_fitness_overall = best_fitnesses[best_index]

    return {
        "best_fitness": best_fitness_overall,
        "best_individual": best_overall,
        "all_fitness": best_fitnesses,
        "best_individuals_all_runs": best_individuals,
        "logbook": logbook
    }


Running setup1 and setups2 for GE and GP

In [26]:

ge_setup1 = run_ge("GE Setup 1", pop_size=500, p_crossover=0.9, p_mutation=0.1, generations=50,toolbox=toolbox_ge)
ge_setup2 = run_ge("GE Setup 2", pop_size=500, p_crossover=0.5, p_mutation=0.3, generations=50,toolbox=toolbox_ge)

gp_setup1=run_gp("GP Setup 1", pop_size=500, p_crossover=0.9, p_mutation=0.1, generations=50,toolbox=toolbox_gp)
gp_setup2=run_gp("GP Setup 2", pop_size=500, p_crossover=0.5, p_mutation=0.3, generations=50,toolbox=toolbox_gp)



===== Running GE Setup 1 =====

Currently on run 1 of 5
gen = 0 , Best fitness = (np.float64(8.0),)
gen = 1 , Best fitness = (np.float64(8.0),) , Number of invalids = 8
gen = 2 , Best fitness = (np.float64(8.0),) , Number of invalids = 3
gen = 3 , Best fitness = (np.float64(8.0),) , Number of invalids = 8
gen = 4 , Best fitness = (np.float64(8.0),) , Number of invalids = 3
gen = 5 , Best fitness = (np.float64(8.0),) , Number of invalids = 11
gen = 6 , Best fitness = (np.float64(8.0),) , Number of invalids = 6
gen = 7 , Best fitness = (np.float64(8.0),) , Number of invalids = 3
gen = 8 , Best fitness = (np.float64(8.0),) , Number of invalids = 2
gen = 9 , Best fitness = (np.float64(8.0),) , Number of invalids = 5
gen = 10 , Best fitness = (np.float64(8.0),) , Number of invalids = 8
gen = 11 , Best fitness = (np.float64(8.0),) , Number of invalids = 8
gen = 12 , Best fitness = (np.float64(8.0),) , Number of invalids = 3
gen = 13 , Best fitness = (np.float64(8.0),) , Number of invalids =

**-----------------------------------Comparisons-----------------------------------------**

Results for both setups

In [27]:
print("\n-----COMPARISON-----\n")

print("GP Setup 1 Best Fitness:", gp_setup1["best_fitness"])
print("Average GP Setup 1 fitness over all runs:", np.mean(gp_setup1["all_fitness"]))
print("GP Setup 1 Best Individual:", gp_setup1["best_individuals"])
print("GP Setup 2 Best Fitness:", gp_setup2["best_fitness"])
print("GP Setup 2 Best Individual:", gp_setup2["best_individuals"])
print("Average GP Setup 2 fitness over all runs:", np.mean(gp_setup2["all_fitness"]))

print("\nGE Setup 1 Best Fitness:", ge_setup1["best_fitness"])
print("GE Setup 1 Best Individual:", ge_setup1["best_individual"])
print("Average GE Setup 1 fitness over all runs:", np.mean(ge_setup1["all_fitness"]))
print("GE Setup 2 Best Fitness:", ge_setup2["best_fitness"])
print("GE Setup 2 Best Individual:", ge_setup2["best_individual"])
print("Average Ge Setup 2 fitness over all runs:", np.mean(ge_setup2["all_fitness"]))



-----COMPARISON-----

GP Setup 1 Best Fitness: 3.0
Average GP Setup 1 fitness over all runs: 3.4
GP Setup 1 Best Individual: nand_(nor_(or_(and_(in_0, in_2), and_(and_(in_3, in_1), and_(in_3, 1))), nor_(or_(in_3, in_1), nor_(and_(in_2, in_2), nor_(nor_(in_0, in_2), and_(in_3, or_(and_(nor_(in_0, in_2), and_(in_0, and_(or_(and_(nand_(nor_(and_(in_0, in_2), in_2), or_(in_1, in_3)), nor_(in_2, nand_(in_3, nor_(in_0, in_0)))), and_(in_3, in_2)), in_1))), nor_(and_(0, nand_(1, and_(or_(and_(0, in_0), or_(nand_(or_(nand_(in_0, nand_(1, 0)), in_1), nand_(0, in_0)), nor_(in_0, in_2))), and_(nor_(in_3, in_1), in_2)))), in_0))))))), nand_(or_(in_1, and_(1, nand_(0, in_0))), nor_(nand_(nor_(nor_(nor_(in_1, in_1), nand_(in_2, in_0)), in_0), or_(in_1, in_3)), in_2)))
GP Setup 2 Best Fitness: 2.0
GP Setup 2 Best Individual: nand_(or_(nand_(nor_(or_(in_2, in_2), 0), nand_(nand_(in_3, in_2), nor_(and_(nand_(in_3, 1), nand_(1, in_2)), nand_(nand_(in_1, in_0), or_(in_0, in_1))))), in_2), or_(nor_(or_(n

Setup's comparison within GP and GE

In [28]:
print("comparing GP setup 1and 2 :-")
if gp_setup1["best_fitness"] < gp_setup2["best_fitness"]:
    print("GP Setup 1 performed better")
else:
    print("GP Setup 2 performed better")
print("\ncomparing GE setup 1and 2 :-")
if ge_setup1["best_fitness"] < ge_setup2["best_fitness"]:
    print("GE Setup 1 performed better")
else:
    print("GE Setup 2 performed better")


comparing GP setup 1and 2 :-
GP Setup 2 performed better

comparing GE setup 1and 2 :-
GE Setup 2 performed better


Cpmparison between GP and GE

In [29]:
def compare(gp_setup, ge_setup, setup_name="Setup 1"):
    print(f"\nDirect GP vs GE Comparison on : {setup_name}")
    print(f"GP Best Fitness: {gp_setup['best_fitness']}")
    print(f"GE Best Fitness: {ge_setup['best_fitness']}")
    if gp_setup['best_fitness'] < ge_setup['best_fitness']:
        print(f"GP {setup_name} performs better than GE {setup_name}")
    elif gp_setup['best_fitnesses'] > ge_setup['best_fitness']:
        print(f"GE {setup_name} performs better than GP {setup_name}")
    else:
        print(f"Both GP and GE {setup_name} perform equally")
compare(gp_setup1, ge_setup1, "Setup 1")
compare(gp_setup2, ge_setup2, "Setup 2")


Direct GP vs GE Comparison on : Setup 1
GP Best Fitness: 3.0
GE Best Fitness: 8.0
GP Setup 1 performs better than GE Setup 1

Direct GP vs GE Comparison on : Setup 2
GP Best Fitness: 2.0
GE Best Fitness: 8.0
GP Setup 2 performs better than GE Setup 2
